In [2]:
import matplotlib.pyplot as plt
import numpy as np
import time
import csv
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, LSTM
np.random.seed(2017)

In [3]:
path_to_dataset='bike_rnn.csv'
sequence_length=20

In [4]:
max_values = 45949
with open(path_to_dataset) as f:
    data = csv.reader(f, delimiter=",")
    bike = []
    nb_of_values = 0
    for line in data:
        try:
            bike.append(float(line[0]))
            nb_of_values += 1
        except ValueError:
            pass
        # 如果大于等于最大行数就退出
        if nb_of_values >= max_values:
            break
    f.close()

In [4]:
bike

[0.0,
 1.0,
 1.0,
 2.0,
 4.0,
 5.0,
 5.0,
 5.0,
 4.0,
 3.0,
 3.0,
 1.0,
 1.0,
 2.0,
 2.0,
 2.0,
 2.0,
 2.0,
 1.0,
 2.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 3.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 2.0,
 1.0,
 1.0,
 1.0,
 1.0,
 2.0,
 2.0,
 2.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 2.0,
 2.0,
 3.0,
 2.0,
 4.0,
 4.0,
 2.0,
 2.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 2.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 2.0,
 2.0,
 1.0,
 2.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 2.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 3.0,
 3.0,
 3.0,
 2.0,
 2.0,
 2.0,
 1.0,
 0.0,
 0.0,
 0.0

In [5]:
result = []
for index in range(len(bike) - sequence_length):
    result.append(bike[index: index + sequence_length])
result = np.array(result)

In [6]:
result[0]

array([0., 1., 1., 2., 4., 5., 5., 5., 4., 3., 3., 1., 1., 2., 2., 2., 2.,
       2., 1., 2.])

In [ ]:
result[1]

In [7]:
row = int(round(0.9 * result.shape[0]))

In [15]:
train = result[:row, :]
np.random.shuffle(train)
X_train = train[:, :-1]
y_train = train[:, -1]
X_test = result[row:, :-1]
y_test = result[row:, -1]

In [35]:
X_train

array([[[10.],
        [11.],
        [11.],
        ...,
        [ 8.],
        [ 9.],
        [ 9.]],

       [[10.],
        [10.],
        [10.],
        ...,
        [10.],
        [10.],
        [10.]],

       [[ 3.],
        [ 3.],
        [ 3.],
        ...,
        [ 9.],
        [ 9.],
        [ 8.]],

       ...,

       [[ 1.],
        [ 1.],
        [ 1.],
        ...,
        [ 0.],
        [ 1.],
        [ 1.]],

       [[ 3.],
        [ 3.],
        [ 3.],
        ...,
        [ 4.],
        [ 4.],
        [ 4.]],

       [[ 2.],
        [ 2.],
        [ 2.],
        ...,
        [ 3.],
        [ 3.],
        [ 3.]]])

In [36]:
X_train.shape

(41336, 19, 1)

In [32]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [34]:
X_train.shape

(41336, 19, 1)

In [ ]:
model = Sequential()
model.add(CuDNNLSTM(128, input_shape=(train_x.shape[1:]), return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())  #normalizes activation outputs, same reason you want to normalize your input data.

model.add(CuDNNLSTM(128, return_sequences=True))
model.add(Dropout(0.1))
model.add(BatchNormalization())

model.add(CuDNNLSTM(128))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(2, activation='softmax'))